In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

# Experimental distance-frequency matrix

In [ ]:
from simulation import get_setup

distance = 0

fig = plt.figure()
fig.set_size_inches(5, 5)
for yaw_deg, marker in zip([0, 30], ["o", "x"]):
    source, mic_positions = get_setup(distance_cm=distance, yaw_deg=yaw_deg)
    source_image = [source[0], -source[1]]
    for i, mic in enumerate(mic_positions):
        plt.scatter(*mic, label=f"mic{i}, {yaw_deg}deg", marker=marker, color=f"C{i}")
plt.plot([4.8, 5.4], [0, 0], color="k", label="wall")
plt.title(f"setup for distance={distance}cm")
plt.legend(loc="upper right")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.axis("equal")

# fig.savefig(f'plots/setup_{distance}.pdf', bbox_inches='tight')

In [ ]:
# exp_name = '2021_02_09_wall_tukey';
# exp_name = '2021_02_09_wall';
#exp_name = "2021_02_23_wall"
exp_name = "2021_02_25_wall"
# exp_name = '2021_02_19_windows';
# exp_name = '2020_12_9_rotating';
# exp_name = '2020_11_26_wall';
# fname = f'results/{exp_name}_real.pkl'
fname = f"../experiments/{exp_name}/all_data.pkl"
# fname = f'../experiments/{exp_name}/battery_data.pkl' # for 2021_02_09_wall, battery study

try:
    df_all = pd.read_pickle(fname)
    print("read", fname)
except Exception as e:
    print(e)
    print("Error: run wall_analysis.py to parse experiments.")
df_all.iloc[:, :8]

# plot_spectrograms(df_freq)

# 0. Debug parsing

In [ ]:
from bin_selection import generate_sweep
from crazyflie_description_py.parameters import N_BUFFER, FS
from dataset_parameters import kwargs_datasets
from pandas_utils import filter_by_dicts
from plotting_tools import *
import seaborn as sns
from wall_detector import WallDetector, normalized_std

ymin = 1e-4
ymax = 3e2
if len(df_all.distance.unique()) > 1:
    distance = 20
else:
    distance = df_all.iloc[0].distance
degree = 0

chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    #"motors": "all45000",
    #"mic_type": "measurement"
    #"mic_type": "audio_deck"
}

df_filtered = filter_by_dicts(df_all, [chosen_dict])
print("plotting\n", df_filtered.iloc[:, :8])

verbose = False

fig_slice, ax_slice = plt.subplots()
fig_slice.set_size_inches(5, 5)
    
for mic_type, df_mic_types in df_filtered.groupby("mic_type"):
    if mic_type == "measurement":
        mic = 0
    else:
        mic = 1
        
    for j, (i_row, row) in enumerate(df_mic_types.iterrows()):
        if (mic_type == "measurement") and (row.snr == 1):  # this is no "real snr"
            continue

        title = f'{row.mic_type} {row.motors} {row.appendix.replace("_", "")} {row.distance} mic{mic}'

        # processing
        wall_detector = WallDetector(exp_name=exp_name, mic_type=row.mic_type)
        try:
            spec_masked_all, freqs_masked = wall_detector.fill_from_row(row)
        except:
            print("skipping", row)
            continue
        wall_detector.remove_bad_freqs(verbose=verbose, dryrun=False)
        wall_detector.merge_close_freqs(verbose=verbose)
        wall_detector.remove_spurious_freqs(verbose=verbose)

        # plotting
        spec_masked = np.mean(spec_masked_all, axis=1)
        fig, ax = plt.subplots()
        fig.set_size_inches(5, 5)
        ax.pcolorfast(row.seconds, freqs_masked, np.log10(spec_masked[:-1, :-1]))
        ax.set_xlim(
            kwargs_datasets[exp_name][row.mic_type]["min_time"],
            kwargs_datasets[exp_name][row.mic_type]["max_time"],
        )
        ax.set_ylim(
            kwargs_datasets[exp_name][row.mic_type]["min_freq"],
            kwargs_datasets[exp_name][row.mic_type]["max_freq"],
        )
        ax.set_xlabel("time [s]")
        ax.set_ylabel("frequency [Hz]")
        save_fig(
            fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_spec.png"
        )

        df_mic = wall_detector.df[wall_detector.df.mic == mic]
        
        label = f"{mic_type}, motors={row.motors}"
        stds = df_mic.groupby("frequency", sort=True).magnitude.apply(normalized_std).values
        #stds = df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanstd).values
        medians = df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanmedian).values
        stds = np.r_[[np.zeros(len(medians))], [2*stds]]
        ax_slice.errorbar(x=np.sort(df_mic.frequency.unique()), 
                          y=medians, 
                          yerr=stds, 
                          label=label, marker='o', markersize=4.0, lw=2.0)
        #sns.scatterplot(
        #    data=df_mic, x="frequency", y="magnitude", hue="counter", ax=ax, linewidth=0
        #)

        fig, ax = plot_raw_signals(spec_masked_all, freqs_masked, mic_idx=mic)
        ax.set_ylim(ymin, ymax)
        ax.set_title(title)
        save_fig(
            fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_raw.png"
        )
        
        normalized_stds = df_mic.groupby("frequency", sort=True)["magnitude"].apply(normalized_std).values
        frequencies = np.sort(df_mic.frequency.unique())
        fig, ax = plt.subplots()
        ax.plot(frequencies, normalized_stds, marker='o')
        ax.set_title(title)
        
ax_slice.set_xlabel("frequency [Hz]")
ax_slice.set_yscale("log")
#ax_slice.set_ylim(1e-1, 1e2)
ax_slice.grid(which="both")
ax_slice.set_title(title)
ax_slice.legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])
save_fig(fig, f"plots/{exp_name}{row.appendix}_slice.png")

# Full analysis


- [x] fix calibration_offline scheme: here we should compare to a different theoretical distribution! 
- [x] add cosine cost function to calibrations
- [x] implement parametric calibration
- [ ] choose frequency bins based on std or distance slice performance
- [ ] add plots of "ambiguous" distances
- [ ] implement "offline calibration" for measurement mics


# 1. Choose dataset

In [ ]:
from wall_analysis import get_df_matrices, FILTERS

exp_name = "2021_02_25_wall"

fname = "results/wall_analysis.pkl"
# method = 'dryrun'
# method = 'new'
method = 'replace'
#method = "read"

if method == "read":
    results_df = pd.read_pickle(fname)
elif method == "replace":
    from pandas_utils import fill_df

    FILTERS = ["mic_type", "snr", "motors"]
    results_df = pd.read_pickle(fname)

    print("read", fname)
    results_df_new = get_df_matrices(exp_name)

    for chosen_tuple, df in results_df_new.groupby(FILTERS):
        assert len(df) == 1
        row = df.iloc[0]

        filter_dict = dict(zip(FILTERS, chosen_tuple))
        filter_dict.update({"exp_name": exp_name})
        fill_dict = {
            "df_dist": row.df_dist,
            "df_freq": row.df_freq,
            "df_matrix": row.df_matrix,
        }
        print("filling results_df_old...")
        results_df = fill_df(results_df, filter_dict, fill_dict)
    pd.to_pickle(results_df, fname)
    print("saved as", fname)
elif method == "new":
    results_df = get_df_matrices(exp_name)
    pd.to_pickle(results_df, fname)
    print("saved as", fname)
elif method == "dryrun":
    results_df = get_df_matrices(exp_name, max_distance=10, plot=True)
results_df

In [ ]:
from pandas_utils import filter_by_dicts

#exp_name = "2021_02_23_wall"
exp_name = "2021_02_25_wall"

# mic_type = 'measurement'
mic_type = "audio_deck"
motors = 'all45000'

filter_dict = dict(exp_name=exp_name, motors=motors, mic_type=mic_type)

df = filter_by_dicts(results_df, [filter_dict])
assert len(df) == 1
row = df.iloc[0]
row

# 2. Choose normalization scheme

In [ ]:
from calibration import get_calibration_function
from wall_detector import normalize_df_matrix
from copy import deepcopy
from simulation import get_df_theory

calib_function = get_calibration_function(plot=False)

max_distance = 50
distances_grid = np.arange(0, max_distance + 1)
distances_idx = np.where(distances_grid[:, None] == row.df_dist[None, :])[0]

np.testing.assert_allclose(distances_grid[distances_idx], row.df_dist)

if row.mic_type == "measurement":
    df_theory = get_df_theory(row.df_freq, distances_grid, chosen_mics=[1])
else:
    df_theory = get_df_theory(row.df_freq, distances_grid, chosen_mics=range(4))

In [ ]:
from wall_detector import prune_df_matrix

df_matrix, df_freq, indices = prune_df_matrix(
    row.df_matrix, row.df_freq, ratio_missing_allowed=0.5, verbose=True
)
df_theory_pruned = df_theory[:, indices, :]

In [ ]:
def cost_period_all(i, plot_d=False, plot_f=True):
    if np.any(np.isnan(df_matrix[:, i, :])):
        print("skipping", df_freq[i])
        return [np.nan] * df_matrix.shape[0]

    if plot_d:
        fig, axs = plt.subplots(1, df_matrix.shape[0], squeeze=False, sharey=True)
        fig.set_size_inches(15, 5)
        fig.suptitle(f"frequency {f:.0f}Hz")
    if plot_f:
        fig_f, axs_f = plt.subplots(1, df_matrix.shape[0], squeeze=False, sharey=True)
        fig_f.set_size_inches(15, 5)
        fig_f.suptitle(f"frequency {f:.0f}Hz")

    costs = []
    for m in range(df_matrix.shape[0]):
        slice_exp = deepcopy(df_matrix[m, i, :])
        slice_the_norm = deepcopy(df_theory_pruned[m, i, :])

        slice_the_norm -= np.min(slice_the_norm)
        slice_the_norm /= np.max(slice_the_norm) - np.min(slice_the_norm)
        slice_the_norm *= np.max(slice_exp) - np.min(slice_exp)
        slice_the_norm += np.min(slice_exp)

        if plot_d:
            axs[0, m].plot(row.df_dist, slice_exp)
            axs[0, m].plot(distances_grid, slice_the_norm)
            axs[0, m].set_title(f"mic{m}")

        f_exp = np.fft.rfftfreq(len(slice_exp), d=1 / len(slice_exp))
        slice_exp -= np.mean(slice_exp)
        fft_exp = np.abs(np.fft.rfft(slice_exp))
        max_exp = f_exp[np.argmax(fft_exp)]

        f_theory = np.fft.rfftfreq(len(slice_the_norm), d=1 / len(slice_exp))
        slice_the_norm -= np.mean(slice_the_norm)
        fft_theory = np.abs(np.fft.rfft(slice_the_norm))
        max_theory = f_theory[np.argmax(fft_theory)]

        costs.append(np.abs(max_exp - max_theory))

        if plot_f:
            axs_f[0, m].plot(f_exp, fft_exp)
            axs_f[0, m].plot(f_theory, fft_theory)
            axs_f[0, m].set_title(f"mic{m}")
    return costs


cost_list = []
for i, f in enumerate(df_freq):
    cost_list.append(cost_period_all(i, plot_d=True, plot_f=True))
cost_arr = np.array(cost_list)

In [ ]:
n_mics = cost_arr.shape[1]

fig, axs = plt.subplots(1, n_mics, sharey=True, squeeze=False)
fig.set_size_inches(10, 5)

fig, ax_cdf = plt.subplots()
fig.set_size_inches(10, 5)

for m in range(n_mics):
    axs[0, m].plot(df_freq, cost_arr[:, m], label=f"mic{m}", marker="o", color=f"C{m}")
    axs[0, m].legend()
    ax_cdf.plot(
        np.sort(cost_arr[:, m]),
        np.linspace(0, 1, cost_arr.shape[0]),
        label=f"mic{m}",
        color=f"C{m}",
    )
axs[0, 0].set_ylabel("cost")
ax_cdf.legend()
ax_cdf.set_ylabel("cdf")
ax_cdf.set_xlabel("absolute peak error")
ax_cdf.grid(which="both")

good_frequencies = df_freq[np.all(cost_arr < 2, axis=1)]
bad_frequencies = df_freq[np.any(cost_arr >= 2, axis=1)]
print("good frequencies:", good_frequencies)
print("bad frequencies:", bad_frequencies)

In [ ]:
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline
from numpy.polynomial import Chebyshev, Legendre, Polynomial, Laguerre

spline_k = 2
spline_ext = 3

for mic_idx in range(row.df_matrix.shape[0]):
    df_norm, values = normalize_df_matrix(
        df_matrix=row.df_matrix, freqs=row.df_freq, method="calibration-online"
    )

    df_norm, df_freq, indices = prune_df_matrix(df_norm, row.df_freq)
    values = values[:, indices]

    xvalues = df_freq
    yvalues = np.log10(values[mic_idx, :, 0])

    plt.figure()
    plt.plot(xvalues, yvalues, label="raw", color="C0", marker="*")
    for i, poly in enumerate([Legendre]):  
        c = poly.fit(xvalues, yvalues, deg=10)
        x, y = c.linspace()
        plt.plot(x, y, label=poly.__name__, color=f"C{i+1}")

    # spline = UnivariateSpline(xvalues, yvalues, k=2)
    # plt.plot(xvalues, spline(xvalues), label='Spline', color='C5')

    knots = np.linspace(min(df_freq), max(df_freq), 7)[1:-1]
    spline = LSQUnivariateSpline(xvalues, yvalues, t=knots, k=spline_k, ext=spline_ext)
    plt.plot(xvalues, spline(xvalues), label="LSQSpline", color=f"C{i+2}", marker="o")
    [plt.axvline(k, color=f"C{i+2}") for k in knots]
    plt.legend()
    plt.title(f"mic{mic_idx}")

In [ ]:
def spline_interpolation(xvalues, df_matrix):
    """ Generate spline interpolation for all distances.
    
    :return calib_values: mics x freqs x distances, interpolated spline curve
    """
    knots = np.linspace(min(df_freq), max(df_freq), 6)[1:-1]
    calib_values = np.empty_like(df_matrix)
    for i, dist in enumerate(row.df_dist):
        for m in range(df_matrix.shape[0]):

            slice_f = np.log10(df_matrix[m, :, i])
            mask = ~np.isnan(slice_f)

            spline = LSQUnivariateSpline(
                xvalues[mask], slice_f[mask], t=knots, k=spline_k, ext=spline_ext
            )
            calib_values[m, :, i] = 10 ** spline(xvalues)
    return calib_values


df_matrix, df_freq, __ = prune_df_matrix(row.df_matrix, row.df_freq)
calib_values = spline_interpolation(df_freq, df_matrix)

fig, axs = plt.subplots(1, calib_values.shape[0], squeeze=False, sharey=True)
fig.set_size_inches(10, 5)
cmap = plt.get_cmap("inferno")
for m in range(calib_values.shape[0]):
    for i, dist in enumerate(row.df_dist):
        axs[0, m].semilogy(
            df_freq, calib_values[m, :, i], color=cmap(i / len(row.df_dist))
        )
    axs[0, m].semilogy(
        df_freq, calib_values[m, :, 0], color=cmap(0), label=f"{row.df_dist[0]}cm"
    )
    axs[0, m].semilogy(
        df_freq,
        calib_values[m, :, i],
        color=cmap(i / len(row.df_dist)),
        label=f"{row.df_dist[-1]}cm",
    )
    axs[0, m].set_title(f"mic{m}")
    axs[0, m].legend(loc="lower right")

In [ ]:
min_value = np.inf
max_value = -np.inf

results = pd.DataFrame(columns=["normalization", "matrix", "values"])
method_dict = dict(
    zip(
        [
            "raw",
            "normalize",
            "calibration-offline",
            "spline",
            "calibration-online",
            "standardize",
            "zero_mean",
            "theoretical",
        ],
        [
            "",
            "normalize",
            calib_function,
            calib_values,
            "calibration-online",
            "standardize",
            "zero_mean",
            "theoretical",
        ],
    )
)

if row.mic_type == "measurement":
    plot_channel_idx = 0  # only choice
else:
    plot_channel_idx = 1  # corresponds to mic below measurement mic

__, df_freq, indices = prune_df_matrix(row.df_matrix, row.df_freq)
df_theory_pruned = df_theory[:, indices, :]

for j, (key, method) in enumerate(method_dict.items()):
    df_matrix = row.df_matrix

    values = None
    df_matrix = df_matrix[:, indices, :]

    if key == "raw":
        df_norm = deepcopy(df_matrix)
    elif key == "theoretical":
        df_norm = deepcopy(df_theory_pruned[:, :, distances_idx])
    else:
        df_norm, values = normalize_df_matrix(
            df_matrix=df_matrix, freqs=df_freq, method=method
        )
    results.loc[len(results), :] = {
        "normalization": key,
        "matrix": df_norm,
        "values": values,
    }
    # print(key, data_type, df_norm.shape)
    min_value = min(min_value, abs(np.min(df_norm)))
    max_value = max(max_value, abs(np.max(df_norm)))
print("range:", min_value, max_value)

In [ ]:
min_freq = min(df_freq)
max_freq = max(df_freq)

df_the = df_theory_pruned[plot_channel_idx]

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
ax, im = plot_df_matrix(
    row.df_dist,
    df_freq,
    df_the[:, distances_idx],
    ax=ax,
    min_freq=min_freq,
    max_freq=max_freq,
    vmin=min_value,
    vmax=max_value,
)
add_colorbar(fig, ax, im)
ax.set_title(f"theoretical")

for normalization, df in results.groupby("normalization", sort=False):
    if normalization == "raw":
        continue

    df_exp = df.iloc[0].matrix[plot_channel_idx]

    min_value = np.min(df_the)
    max_value = 2 * np.max(df_the)

    # print(np.all(np.isnan(df_exp), axis=1))

    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    # plot_df_matrix(row.df_dist, row.df_freq, df_norm[i], ax=axs[0, i], min_freq=min_freq, max_freq=max_freq, vmin=1e-3, vmax=3)
    ax, im = plot_df_matrix(
        row.df_dist,
        df_freq,
        df_exp,
        ax=ax,
        min_freq=min_freq,
        max_freq=max_freq,
        vmin=min_value,
        vmax=max_value,
    )
    add_colorbar(fig, ax, im)
    ax.set_title(f"{normalization}, experimental")

# print(np.nanmin(row.df_matrix), np.nanmax(row.df_matrix))
# print(row.df_matrix.shape)
# print(row.df_matrix[0, :, 0])

In [ ]:
# theoretical vs. measured: matrices
df_exp = results.loc[results.normalization == "raw"].iloc[0].matrix[plot_channel_idx]

fig, axs = plt.subplots(1, 2, squeeze=False, sharey=True)
[ax.set_xlabel("distance [cm]") for ax in axs.flatten()]
axs[0, 0].set_title(row.mic_type + " mic")
axs[0, 1].set_title("theoretical")

min_freq = min(df_freq)
max_freq = max(df_freq)

fig.set_size_inches(10, 5)
# plot_df_matrix(row.df_dist, row.df_freq, df_norm[i], ax=axs[0, i], min_freq=min_freq, max_freq=max_freq, vmin=1e-3, vmax=3)
ax, im = plot_df_matrix(
    row.df_dist, df_freq, df_exp, ax=axs[0, 0], min_freq=min_freq, max_freq=max_freq
)
add_colorbar(fig, ax, im)
ax, im = plot_df_matrix(
    row.df_dist,
    df_freq,
    df_the[:, distances_idx],
    ax=axs[0, 1],
    min_freq=min_freq,
    max_freq=max_freq,
)
add_colorbar(fig, ax, im)
save_fig(fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}_matrices.png")

In [ ]:
# calibration plots

distance = 6
print(row.df_dist)
distance_idx = np.where(row.df_dist == distance)[0]
print(distance_idx)

print(df_freq.shape)
df_exp = results.loc[results.normalization == "raw"].iloc[0].matrix[plot_channel_idx]
print(df_exp.shape)
slice_exp = df_exp[:, distance_idx]
print(slice_exp.shape)

fig, axs = plt.subplots(1, 2, squeeze=False)
fig.set_size_inches(10, 5)

axs[0, 0].plot(df_freq, slice_exp, label="raw")

axs[0, 1].plot(df_freq, slice_exp, label="raw")

# plot_list = ['calibration-online', 'calibration-offline', 'spline', 'standardize']
plot_list = ["calibration-online", "standardize"]

for normalization, df in results.groupby("normalization", sort=False):
    if not normalization in plot_list:
        continue
    row_exp = df.iloc[0]
    print(normalization, row_exp["values"].shape, len(df_freq))

    if normalization == "spline":
        # choose one of the spline calibration curves
        values = row_exp["values"][plot_channel_idx, :, distance_idx[0]]
    else:
        # otherwise there is only one calibration curve
        values = row_exp["values"][plot_channel_idx, :, 0]
    axs[0, 0].plot(df_freq, values, label=normalization)

    df_exp = row_exp.matrix[plot_channel_idx]
    slice_exp = df_exp[:, distance_idx]
    axs[0, 1].plot(df_freq, slice_exp, label=normalization)

slice_the = df_the[:, distances_idx][:, distance_idx]
axs[0, 1].plot(df_freq, slice_the, label="theoretial")

axs[0, 0].legend(loc="lower right")
axs[0, 1].legend(loc="lower right")

axs[0, 0].set_title("$|Y(f)|^2$")
axs[0, 1].set_title("$|H(f)|^2$")

axs[0, 0].set_xlabel("frequency [Hz]")
axs[0, 1].set_xlabel("frequency [Hz]")

# axs[0, 0].set_yscale('log')
# axs[0, 1].set_yscale('log')
# axs[0, 0].set_ylim(1e-2, 1e2)
# axs[0, 1].set_ylim(1e-2, 1e2)

axs[0, 0].grid()
axs[0, 1].grid()
save_fig(fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}_calibration.png")

# 3. Visualize cost function

In [ ]:
from simulation import get_freq_slice_theory
from wall_detector import get_probability_cost, get_probability_fft

mic_idx = 1

# chosen_methods = results.normalization.unique()
chosen_methods = ["raw"] + plot_list + ["theoretical"]

chosen_frequencies = df_freq
# chosen_frequencies = good_frequencies

freq_indices = np.where(chosen_frequencies[None, :] == df_freq[:, None])[0]
np.testing.assert_allclose(df_freq[freq_indices], chosen_frequencies)

absolute_err_dict = {
    key: {method: [] for method in chosen_methods} for key in ["cost", "fft"]
}

distances = row.df_dist[row.df_dist <= max_distance]
for i_d, distance in enumerate(distances):

    fig, ax = plt.subplots()
    fig_cost, ax_cost = plt.subplots()

    for i_method, method in enumerate(chosen_methods):
        df = results.loc[results.normalization == method]
        slice_exp = df.iloc[0].matrix[[mic_idx], freq_indices, i_d]

        # doing this here for plotting reasons. Doesn't change performance as
        # this is done again in get_probability_cost
        slice_exp -= np.mean(slice_exp)
        slice_exp /= np.std(slice_exp)

        proba_cost = get_probability_cost(
            slice_exp, df_freq[freq_indices], 
            distances_grid, mic_idx=mic_idx
        )
        distances_fft, proba_fft = get_probability_fft(
            slice_exp, 
            df_freq[freq_indices]
        )

        # algorithm
        d_cost_idx = np.argmax(proba_cost)
        d_cost = distances_grid[d_cost_idx]
        absolute_err_dict["cost"][method].append(abs(d_cost - distance))

        d_fft_idx = np.argmax(proba_fft)
        d_fft = distances_fft[d_fft_idx]
        absolute_err_dict["fft"][method].append(abs(d_fft - distance))

        # plotting
        ax.plot(chosen_frequencies, slice_exp, label=method, color=f"C{i_method}")
        ax.set_xlim(min_freq, max_freq)
        ax.set_ylim(-3, 3)
        ax.set_xlabel("frequency [Hz]")
        ax.set_ylabel("amplitude")
        ax.set_title(f"slices for distance {distance:.0f}cm")

        ax_cost.semilogy(distances_grid, proba_cost, color=f"C{i_method}", label=method)
        ax_cost.semilogy(distances_fft, proba_fft, color=f"C{i_method}", ls=":")
        ax_cost.axvline(x=d_cost, color=f"C{i_method}")
        ax_cost.axvline(x=d_fft, color=f"C{i_method}", ls=":")

        ax_cost.axvline(x=distance, color="black", ls=":")
        ax_cost.set_xlabel("distance sweep [cm]")
        ax_cost.set_ylabel("probability")
        ax_cost.set_xlim(min(distances_grid), max(distances_grid))
        ax_cost.set_ylim(1e-5, 1)
        ax_cost.set_title(f"probability for distance {distance:.0f}cm")

    ax.legend(loc="upper left")
    ax_cost.legend(loc="lower right")

    # fname = f'results/{exp_name}/slice_{mic_type}_{distance:.0f}_snr{snr}.png'
    # save_fig(fig, fname)
    #
    # fname = f'results/{exp_name}/cost_{mic_type}_{distance:.0f}_snr{snr}.png'
    # save_fig(fig_cost, fname)

# 4. Evaluate algorithm performance

In [ ]:
from matplotlib.lines import Line2D

markers = [m for m in list(Line2D.markers.keys()) if m not in [".", "", ","]]

for key, absolute_err in absolute_err_dict.items():
    i = 0

    fig, axs = plt.subplots(1, 2, squeeze=False)
    fig.set_size_inches(15, 5)
    for method, absolute_errors in absolute_err.items():
        markersize = 8 - i
        axs[0, 0].plot(
            distances,
            absolute_errors,
            label=method,
            marker=markers[i],
            ls=":",
            markersize=markersize,
        )

        xvals = sorted(absolute_errors)
        yvals = np.linspace(0, 1, len(xvals))
        axs[0, 1].plot(
            xvals, yvals, label=method, marker=markers[i], ls=":", markersize=markersize
        )
        i += 1

    # axs[0, 0].set_yscale('log')
    axs[0, 0].set_ylabel("absolute error [cm]")
    axs[0, 0].set_xlabel("distance [cm]")
    axs[0, 0].legend(loc="upper right")

    axs[0, 1].set_ylabel("pdf")
    axs[0, 1].set_xlabel("absolute error [cm]")
    axs[0, 1].set_xlim(-1, max(distances))
    axs[0, 1].grid(which="both")
    axs[0, 1].legend(loc="lower right")

    # fname = f'results/{exp_name}/performance_{mic_type}_snr{snr}_subset.png'
    # fname = f'results/{exp_name}/performance_{mic_type}_snr{snr}_all.png'
    fname = f"plots/{exp_name}_{mic_type}_{key}_performance.png"
    save_fig(fig, fname)